In [1]:
import glob
import pandas as pd
from math import isnan
from copy import deepcopy as copy
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

In [2]:
import os
path = os.getcwd() + '/data' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Find which users are most often retweeted in this dataset

In [83]:
english_frame = frame[frame['language'] == 'English']

In [ ]:
retweeted_users = {}
for row in english_frame[english_frame['tco1_step1'].notnull()]['tco1_step1']:
    if isinstance(row, str) and 'https://twitter.com/' in row:
        row = row[row.index('https://twitter.com/') + 20:].lower()
        user = row[:row.find('/')]
    if user not in retweeted_users:
        retweeted_users[user] = 0
    retweeted_users[user] += 1

In [ ]:
retweeted_users = dict([(key,retweeted_users[key]) for key in retweeted_users if retweeted_users[key] > 1000])

In [ ]:
file = open('retweeted_users.csv','w') 

In [ ]:
for key in retweeted_users:
    file.write(key + ',' + str(retweeted_users[key]) + '\n')
file.close()

### Which hashtags are used most often

In [ ]:
pattern.sub('',s)

In [ ]:
import re, string
pattern = re.compile('[\W_]+')

In [ ]:
hashtags = {}
for row in frame[frame['tco1_step1'].notnull()]['content']:
    if not isinstance(row,str):
        continue
    tags = [part[1:].lower() for part in row.split() if part.startswith('#')]
    for tag in tags:
        tag = pattern.sub('', tag)
        if tag not in hashtags:
            hashtags[tag] = 0
        hashtags[tag] += 1

In [ ]:
len([hashtag for hashtag ])

In [ ]:
import codecs
file = codecs.open('hashtags.csv','w','utf-8') 

In [ ]:
for key in hashtags:
    file.write(key + ',' + str(hashtags[key]) + '\n')
file.close()

### getting sentiment of sentence

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords 
import string
lemma = WordNetLemmatizer()
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 

def clean(doc):
    # input: string
    # output: string
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude or ch == "!")
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()).split()
    link_free = [word for word in normalized if 'http' not in word]
    cleaned = " ".join(word for word in link_free)
    
    return cleaned

In [4]:
import nltk
import string as st
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# getting sentiment of sentence
def sentiment(tweets):
    # input: list of strings
    # output: list of ints
    sentiments = []
    for tweet in tweets:
        tweet = clean(tweet)
        score = sid.polarity_scores(tweet)
        sentiments.append(score['compound'])
    return sentiments

C:\ProgramData\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
interesting = ['author','content','region','language','following','followers','account_type','account_category']
for c in interesting:
    print(c, len(frame[c].unique()))

In [ ]:
for column in frame.columns:
    print(column, len(frame[column].unique()))

In [ ]:
frame[frame['tweet_id']==670004299326361600]['content'].values

In [ ]:
frame.loc[frame['account_category'] == 'Fearmonger'][['tweet_id','content']]

In [5]:
df = frame[frame['language'] == 'English']

In [36]:
tweets['NonEnglish'][list(tweets['NonEnglish'].keys())[0]]
sentiments['NonEnglish'][list(tweets['NonEnglish'].keys())[0]]

[0.0,
 -0.25,
 -0.5994,
 0.6476,
 0.0,
 0.0,
 0.0516,
 0.1027,
 0.0,
 0.0,
 -0.4939,
 0.0,
 -0.2732,
 0.7184,
 0.0,
 0.8429,
 0.0,
 0.6369,
 0.0,
 0.4215,
 0.296,
 0.0]

In [35]:
from tqdm import tqdm
tweets = {}
sentiments = {}
for category in df['account_category'].unique():
    print(category)
    dfc = df[df['account_category'] == category]
    g = dfc.groupby(by = dfc['external_author_id']).groups
    tweets[category] = {}
    sentiments[category] = {}
    for user in tqdm(g):
        tweets[category][user] = df.iloc[g[user]]['content'].values
        sentiments[category][user] = sentiment(tweets[category][user])

RightTroll


100%|████████████████████████████████████████████████████████████████████████████████| 449/449 [05:49<00:00,  1.52it/s]


Fearmonger


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [00:04<00:00, 25.04it/s]


NonEnglish


...

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [04:10<00:00,  2.20s/it]


NewsFeed


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [11:02<00:00, 12.51s/it]


Commercial


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:05<00:00, 13.70s/it]


In [80]:
g = df.groupby('account_category').groups

In [81]:
df.iloc[g['RightTroll']]

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN
5,906000000000000000,10_GOP,"Dan Bongino: ""Nobody trolls liberals better th...",Unknown,English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,247,...,Right,0,RightTroll,0,905874659358453760,914320835325853696,http://twitter.com/905874659358453760/statuses...,https://twitter.com/FoxNews/status/91423949678...,NaN,NaN
6,906000000000000000,10_GOP,🐝🐝🐝 https://t.co/MorL3AQW0z,Unknown,English,10/1/2017 2:48,10/1/2017 2:48,1050,9644,248,...,Right,1,RightTroll,0,905874659358453760,914321156466933760,http://twitter.com/905874659358453760/statuses...,https://twitter.com/Cernovich/status/914314644...,NaN,NaN
7,906000000000000000,10_GOP,'@SenatorMenendez @CarmenYulinCruz Doesn't mat...,Unknown,English,10/1/2017 2:52,10/1/2017 2:53,1050,9644,249,...,Right,0,RightTroll,0,905874659358453760,914322215537119234,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN
8,906000000000000000,10_GOP,"As much as I hate promoting CNN article, here ...",Unknown,English,10/1/2017 3:47,10/1/2017 3:47,1050,9646,250,...,Right,0,RightTroll,0,905874659358453760,914335818503933957,http://twitter.com/905874659358453760/statuses...,http://www.cnn.com/2017/09/27/us/puerto-rico-a...,NaN,NaN
9,906000000000000000,10_GOP,After the 'genocide' remark from San Juan Mayo...,Unknown,English,10/1/2017 3:51,10/1/2017 3:51,1050,9646,251,...,Right,0,RightTroll,0,905874659358453760,914336862730375170,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN


In [76]:
g = df.groupby(by = df['account_category']).groups

In [79]:
df.iloc[g['RightTroll']][df.iloc[g['RightTroll']]['account_category'] != 'RightTroll']

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN
5,906000000000000000,10_GOP,"Dan Bongino: ""Nobody trolls liberals better th...",Unknown,English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,247,...,Right,0,RightTroll,0,905874659358453760,914320835325853696,http://twitter.com/905874659358453760/statuses...,https://twitter.com/FoxNews/status/91423949678...,NaN,NaN
6,906000000000000000,10_GOP,🐝🐝🐝 https://t.co/MorL3AQW0z,Unknown,English,10/1/2017 2:48,10/1/2017 2:48,1050,9644,248,...,Right,1,RightTroll,0,905874659358453760,914321156466933760,http://twitter.com/905874659358453760/statuses...,https://twitter.com/Cernovich/status/914314644...,NaN,NaN
7,906000000000000000,10_GOP,'@SenatorMenendez @CarmenYulinCruz Doesn't mat...,Unknown,English,10/1/2017 2:52,10/1/2017 2:53,1050,9644,249,...,Right,0,RightTroll,0,905874659358453760,914322215537119234,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN
8,906000000000000000,10_GOP,"As much as I hate promoting CNN article, here ...",Unknown,English,10/1/2017 3:47,10/1/2017 3:47,1050,9646,250,...,Right,0,RightTroll,0,905874659358453760,914335818503933957,http://twitter.com/905874659358453760/statuses...,http://www.cnn.com/2017/09/27/us/puerto-rico-a...,NaN,NaN
9,906000000000000000,10_GOP,After the 'genocide' remark from San Juan Mayo...,Unknown,English,10/1/2017 3:51,10/1/2017 3:51,1050,9646,251,...,Right,0,RightTroll,0,905874659358453760,914336862730375170,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN


In [45]:
for asdf in g1:
    print(asdf)
    break

34976398


In [41]:
g1 = df1.groupby(by = df1['external_author_id']).groups

In [25]:
import csv
with open('full_troll_tweet_sentiments.csv', 'w', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['tweet','sentiment','category'])
    for category in df['account_category'].unique():
        for i in tqdm(range(int(len(tweets[category])/10))):
           writer.writerow([tweets[category][i], sentiments[category][i], category])

100%|████████████████████████████████████████████████████████████████████████| 11258/11258 [00:00<00:00, 181966.80it/s]


In [ ]:
category + str(frame['account_category'].value_counts()[category]) + 'tweet_sentiments.csv'

In [ ]:
import csv
for category in df['account_category'].unique():
    fname = str(df['account_category'].value_counts()[category]) +'_'+ category + '_tweet_sentiments.csv'
    with open(fname, 'w', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['tweet','sentiment'])
        for key, value in tweets.items():
           writer.writerow([tweets[key], sentiments[key]])

In [ ]:
frame['account_category'].value_counts()

In [ ]:
len(english_frame)

In [ ]:
import time

start = time.time()
stats = {}
for category in english_frame['account_category'].unique():
    tweets = english_frame[english_frame['account_category'] == category]['content'].values[:10000]
    s = sentiment(tweets)
    stats[category] = {'avg':np.mean(s),'max':np.max(s),'min':np.min(s),'std':np.std(s),'cnt':len(s)}
end = time.time()
print(end-start)

In [ ]:
['category','avg', 'max', 'min', 'std', 'cnt']

In [ ]:
stats

In [ ]:
english_frame

In [ ]:
file = open('category_sentiment.csv','w') 
file.write('category,avg,max,min,std,cnt\n')
for category in stats:
    file.write(category)
    for key in stats[category]:
        file.write(',' + str(stats[category][key]))
    file.write('\n')
file.close()

In [ ]:
tweet = '19,000 RESPECTING our National Anthem! #StandForOurAnthem🇺🇸 https://t.co/czutyGaMQV'

In [ ]:
import re, string
pattern = re.compile('[\W_]+')

In [ ]:
len(english_frame[english_frame['content'].notnull()]['content'].values)

In [ ]:
english_tweets = english_frame[english_frame['content'].notnull()]['content'].values[:100000]

In [ ]:
lda_model = trainLDA(english_tweets)

In [ ]:
from tqdm import tqdm

In [ ]:
tags = {}
j = 0
for tweet in tqdm(english_tweets[:100000]):
    if not isinstance(tweet,str):
        continue

    s = sentiment([tweet])[0]
    hash_tags = set([pattern.sub('',part[1:].lower()) for part in tweet.split() if part.startswith('#')])
    for tag in hash_tags:
        if tag not in tags:
            tags[tag] = []
        tags[tag].append(s)

In [ ]:
new_tags = dict([(key,tags[key]) for key in tags if len(tags[key]) > 500])
print(len(new_tags))

In [ ]:
import codecs
file = codecs.open('bostock_hashtag_sentiment.csv','w','utf-8') 
file.write('carat,price\n')
i = 0
for tag in new_tags:
    for s in new_tags[tag]:
        file.write(str(i) + ',' + str(s) + ',' + tag + '\n')
    i += 1
file.close()